In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
cities_geom = gpd.read_file('../data/us-major-cities/USA_Major_Cities.shp')
og_crs = cities_geom.crs
cols_of_interest = [
    'NAME', # name of city
    'CLASS', # city, town, etc
    'ST', # state code
    'POPULATION',
    'geometry' # Point
]
cities_geom = cities_geom[cols_of_interest]
# drop AK and HI
cities_geom = cities_geom[(cities_geom['ST'] != 'AK') & (cities_geom['ST'] != 'HI')]
cities_geom

In [ ]:
iata = pd.read_csv('../data/airport-codes/mborsetti/airports.csv').dropna()
iata = iata.loc[iata['country'] == 'US']
iata.sort_values('iata')

In [ ]:
state_lookup = pd.read_csv('../data/db1b-survey/lookups/L_STATE_ABR_AVIATION.csv')
iata_cols = [
    'iata',
    'city',
    'subd',
]
iata_state_codes = iata[iata_cols].merge(state_lookup, how='inner', left_on='subd', right_on='Description').drop(columns=['Description'])
iata_state_codes = iata_state_codes.rename(columns={'subd': 'state', 'Code': 'ST'})
iata_state_codes.to_csv('../out/iatastates.csv')
iata_state_codes

In [ ]:
cities_cols = [
    'NAME',
    'POPULATION',
    'geometry',
    'ST'
]
iata_cols = [
    'iata',
    'city',
    'ST',
]
cities_geom = pd.DataFrame(cities_geom)
city_to_iata = cities_geom[cities_cols].merge(
    iata_state_codes[iata_cols], how='inner',
    left_on=['NAME', 'ST'], right_on=['city', 'ST']
).drop(columns='NAME').rename(columns={'POPULATION': 'population', 'ST': 'state_id', 'geometry': 'city_pt'})
city_to_iata

In [ ]:
vols = pd.read_csv('../data/db1b-survey/market/processed/db1b-passenger-vol-2022-full.csv')
vols

In [ ]:
vols_cols = [
    'Year',
    'Origin',
    'Dest',
    'NonStopMiles',
    'Passengers'
]
city_iata_cols = [
    'iata',
    'city',
    'state_id',
    'population',
]

origin_iata = vols[vols_cols].merge(city_to_iata[city_iata_cols], how='inner', left_on='Origin', right_on='iata')\
    .rename(columns={'city': 'city_origin', 'state_id': 'state_origin', 'population': 'pop_origin'})\
        .drop(columns=['iata'])

od_iata = origin_iata.merge(city_to_iata[city_iata_cols], how='inner', left_on='Dest', right_on='iata')\
    .rename(columns={'city': 'city_dest', 'state_id': 'state_dest', 'population': 'pop_dest'})\
        .drop(columns=['iata'])
        
od_iata

In [ ]:
# convert miles to km
mi_to_km_factor = 1.60934
od_iata['NonStopMiles'] *= mi_to_km_factor
od_iata = od_iata.rename(columns={'NonStopMiles': 'NonStopKm'})

# idea see:
# https://www.statista.com/statistics/1185559/carbon-footprint-of-travel-per-kilometer-by-mode-of-transport
# assume 255 g/km of co2 per passenger for flights <1000km, or 156 g/km for flights >=1000km
# units: g/km flown

# todo this should all be replaced with google travel API once we have actual flight data
co2_per_person_short = 255
co2_per_person_long = 156

short_flights_idx = od_iata.loc[lambda row: row['NonStopKm'] < 1000].index
long_flights_idx = od_iata.loc[lambda row: row['NonStopKm'] >= 1000].index

annual_g_co2 = od_iata['NonStopKm'] * od_iata['Passengers']
annual_g_co2[short_flights_idx] *= co2_per_person_short
annual_g_co2[long_flights_idx] *= co2_per_person_long

od_iata['co2_g'] = annual_g_co2
od_iata

In [ ]:
# High speed double track on new stone rail road stone bed -- high cost/km = 1650000
# Install a Centralized traffic control system double track -- high cost/km = 257825
# source: https://compassinternational.net/railroad-engineering-construction-cost-benchmarks/
cost_per_km = 1650000 + 257825
od_iata['construction_cost_usd'] = od_iata['NonStopKm'] * cost_per_km

# assume average HSR travel speed of 200 km/h
hsr_avg_speed = 200
od_iata['hsr_travel_time_hr'] = od_iata['NonStopKm'] / hsr_avg_speed

# assume average flight travel speed of 800 km/h
plane_avg_speed = 800
od_iata['plane_travel_time_hr'] = od_iata['NonStopKm'] / plane_avg_speed
# todo add ~3 hours for travel to airport, security, gate waiting time

od_iata

In [ ]:
od_iata.to_csv('../data/algo_testing_data.csv')